In [1]:
# -*- coding: utf-8 -*-
"""EduTutor_AI_App.ipynb

Educational AI Assistant with Signup, Login, Role, and Language Selection
"""

!pip install transformers torch gradio -q

import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# ---------------- Load model ----------------
model_name = "ibm-granite/granite-3.2-2b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto" if torch.cuda.is_available() else None
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# ---------------- AI Response Generator ----------------
def generate_response(prompt, max_new_tokens=600):
    try:
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024)
        if torch.cuda.is_available():
            inputs = {k: v.to(model.device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                temperature=0.7,
                do_sample=True,
                top_p=0.9,
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id
            )

        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        if response.startswith(prompt):
            response = response[len(prompt):]
        return response.strip()

    except Exception as e:
        return f"⚠️ Error: {str(e)}"

# ---------------- Features ----------------
def concept_explanation(concept, lang="English"):
    return generate_response(f"Explain the concept of {concept} in detail with examples. Language: {lang}", 800)

def quiz_generator(concept, lang="English"):
    return generate_response(f"Generate 5 quiz questions on {concept} with answers. Language: {lang}", 800)

def text_summary(text, lang="English"):
    return generate_response(f"Summarize this text in simple words. Language: {lang}\n\n{text}", 600)

def flashcard_creator(topic, lang="English"):
    return generate_response(f"Create 5 flashcards (Q&A) for topic: {topic}. Language: {lang}", 700)

# ---------------- User Database (in memory) ----------------
USER_DB = {}  # {username: {"password": "...", "role": "..."}}

# ---------------- Auth Functions ----------------
def signup(username, password, role):
    if username in USER_DB:
        return "⚠️ Username already exists. Try another."
    USER_DB[username] = {"password": password, "role": role}
    return f"✅ Signup successful! Please login as {role}."

def login(username, password, role, language):
    if username in USER_DB and USER_DB[username]["password"] == password and USER_DB[username]["role"] == role:
        return (
            gr.update(visible=False),  # hide login/signup
            gr.update(visible=True),   # show main app
            f"✅ Welcome {username}! You are logged in as **{role}**. Language: {language}",
            language
        )
    return (
        gr.update(visible=True),
        gr.update(visible=False),
        "❌ Invalid credentials or role mismatch. Please try again.",
        "English"
    )

# ---------------- Gradio Interface ----------------
with gr.Blocks() as app:
    gr.Markdown("# 📘 EduTutor AI App")

    language_state = gr.State("English")

    # ----------- AUTH PAGES -----------
    with gr.Group(visible=True) as auth_page:
        gr.Markdown("## 🔐 Login / Signup")

        with gr.Tab("Login"):
            username_in = gr.Textbox(label="Username")
            password_in = gr.Textbox(label="Password", type="password")
            role_in = gr.Radio(["Student", "Faculty"], label="Role", value="Student")
            lang_in = gr.Dropdown(["English", "Tamil", "Hindi", "French"], value="English", label="Select Language")
            login_btn = gr.Button("Login 🚀")
            login_msg = gr.Markdown("")

        with gr.Tab("Sign Up"):
            new_user = gr.Textbox(label="Choose Username")
            new_pass = gr.Textbox(label="Choose Password", type="password")
            new_role = gr.Radio(["Student", "Faculty"], label="Role", value="Student")
            signup_btn = gr.Button("Sign Up 📝")
            signup_msg = gr.Markdown("")

    # ----------- MAIN APP (hidden until login) -----------
    with gr.Group(visible=False) as main_app:
        welcome_msg = gr.Markdown("")

        with gr.Tabs():
            with gr.TabItem("📘 Concept Explanation"):
                concept_input = gr.Textbox(label="Enter a concept")
                explain_btn = gr.Button("Explain")
                explanation_output = gr.TextArea(label="Explanation", lines=12)
                explain_btn.click(concept_explanation, [concept_input, language_state], explanation_output)

            with gr.TabItem("📝 Quiz Generator"):
                quiz_input = gr.Textbox(label="Enter a topic")
                quiz_btn = gr.Button("Generate Quiz")
                quiz_output = gr.TextArea(label="Quiz Questions", lines=15)
                quiz_btn.click(quiz_generator, [quiz_input, language_state], quiz_output)

            with gr.TabItem("📄 Summary Generator"):
                summary_input = gr.Textbox(label="Enter text", lines=6)
                summary_btn = gr.Button("Summarize")
                summary_output = gr.TextArea(label="Summary", lines=8)
                summary_btn.click(text_summary, [summary_input, language_state], summary_output)

            with gr.TabItem("🎴 Flashcards"):
                flash_input = gr.Textbox(label="Enter a topic")
                flash_btn = gr.Button("Generate Flashcards")
                flash_output = gr.TextArea(label="Flashcards", lines=12)
                flash_btn.click(flashcard_creator, [flash_input, language_state], flash_output)

    # ----------- BUTTON LOGIC -----------
    signup_btn.click(signup, [new_user, new_pass, new_role], signup_msg)

    login_btn.click(
        login,
        inputs=[username_in, password_in, role_in, lang_in],
        outputs=[auth_page, main_app, login_msg, language_state]
    )

app.launch(share=True)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5d0ebbfafce14e1259.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
